In [ ]:
:dep pcap-file = "2"
:dep serde = { version = "1", features = ["derive"] }
:dep serde_json = "1"

use std::fs::File; 
use std::fmt;

use pcap_file::pcapng::PcapNgReader; 
use pcap_file::pcapng::blocks::Block;
use pcap_file::pcapng::blocks::enhanced_packet::EnhancedPacketBlock;
use serde::{Serialize, Serializer};

In [ ]:
fn serialize_family_hex<S>(val: &Option<FeatureFamily>, s: S) -> Result<S::Ok, S::Error>
where S: serde::Serializer {
    match val {
        Some(family) => {
            let label = match family {
                FeatureFamily::Dsp => format!("{:#04x} : Dsp", 0x96),
                FeatureFamily::MaybeDeviceAckWriteSingleFeature => format!("{:#04x} : MaybeDeviceAckWriteSingleFeature", 0x0a),
                FeatureFamily::Global => format!("{:#04x} : Global", 0x00),
                // Here, we just use the String directly!
                FeatureFamily::ParsingError(raw_hex) => format!("Parsing Error: {}", raw_hex),
            };
            s.serialize_str(&label)
        },
        None => s.serialize_none(),
    }
}

// Serializer for Option<FeatureName>
fn serialize_name_hex<S>(val: &Option<FeatureName>, s: S) -> Result<S::Ok, S::Error>
where S: Serializer {
    match val {
        Some(name) => {
            let hex = match name {
                FeatureName::SurroundToggle   => 0x00,
                FeatureName::SurroundLevel    => 0x01,
                FeatureName::DialogPlusToggle => 0x02,
                FeatureName::DialogPlusLevel  => 0x03,
                FeatureName::SmartVolumeToggle => 0x04,
                FeatureName::SmartVolumeLevel => 0x05,
                FeatureName::SmartVolumeMode  => 0x06,
                FeatureName::CrystalizerToggle => 0x07,
                FeatureName::CrystalizerLevel => 0x08,
                FeatureName::EqToggle         => 0x09,
                FeatureName::EqPreAmp         => 0x0a,
                FeatureName::Eq31Hz           => 0x0b,
                FeatureName::Eq62Hz           => 0x0c,
                FeatureName::Eq125Hz          => 0x0d,
                FeatureName::Eq250Hz          => 0x0e,
                FeatureName::Eq500Hz          => 0x0f,
                FeatureName::Eq1kHz           => 0x10,
                FeatureName::Eq2kHz           => 0x11,
                FeatureName::Eq4kHz           => 0x12,
                FeatureName::Eq8kHz           => 0x13,
                FeatureName::Eq16kHz          => 0x14,
                FeatureName::BassToggle       => 0x18,
                FeatureName::BassLevel        => 0x19,
                FeatureName::ParsingError(e)  => *e,
            };
            s.serialize_str(&format!("{:#04x} : {:?}", hex, name))
        },
        None => s.serialize_none(),
    }
}

fn serialize_feature_value<S>(val: &Option<f32>, s: S) -> Result<S::Ok, S::Error> 
where S: serde::Serializer {
    match val {
        Some(v) => {
            let formatted = format!("{:#010x} ({:.1})", v.to_bits(), v);
            s.serialize_str(&formatted)
        },
        None => {
            s.serialize_none()
        }
    }
}

mod hex_serde {
    use serde::Serializer;
    use std::fmt::LowerHex;

    pub fn serialize<S, T>(val: &T, s: S) -> Result<S::Ok, S::Error>
    where
        S: Serializer,
        T: LowerHex, 
    {
        let format_str = format!("{:#04x}", val);
        s.serialize_str(&format_str)
    }
}


In [ ]:
#[derive(Debug, Serialize)]
struct Packet {
    #[serde(with = "hex_serde")]
    id: u64, 
    typ: UrbType, 
    transfer_type: TransferType, 
    endpoint: UsbEndpoint, 
    #[serde(with = "hex_serde")]
    device_address: u8, 
    #[serde(with = "hex_serde")]
    bus_number: u16, 
    urb_status: UrbStatus, 
    #[serde(with = "hex_serde")]
    transfer_flags: u32, 
    setup_fragment: Option<SetupFragment>,
    data_fragment: Option<DataFragment>, 
}

impl fmt::Debug for Packet {
    fn fmt(&self, f: &mut fmt::Formatter<'_>) -> fmt::Result {
        let value_str = match self.value {
            Some(v) => format!("{:#010x} ({:.1})", v.to_bits(), v),
            None => "None".to_string(),
        };

        f.debug_struct("Feature")
            .field("family", &self.family)
            .field("name", &self.name)
            .field("value", &format_args!("{}", value_str))
            .finish()
    }
}

#[derive(Debug, Serialize)]
enum CommDirection {
    HostIn, 
    HostOut, 
}

#[derive(Debug, Serialize)]
enum UrbType {
    ParsingError, 
    Submit, 
    Complete, 
    UrbError, 
}

#[derive(Debug, Serialize)]
enum TransferType {
    #[serde(with = "hex_serde")]
    ParsingError(u8),
    Isochronous, 
    Interrupt, 
    Control, 
    Bulk, 
}

#[derive(Debug, Serialize)]
struct UsbEndpoint {
    #[serde(with = "hex_serde")]
    raw: u8, 
    direction: CommDirection,
}

#[derive(Debug, Serialize)]
struct SetupFragment {
    #[serde(with = "hex_serde")]
    bmRequestType: u8, 
    #[serde(with = "hex_serde")]
    bRequest: u8,
    #[serde(with = "hex_serde")]
    wValue: u16, 
    #[serde(with = "hex_serde")]
    wIndex: u16, 
    #[serde(with = "hex_serde")]
    wLength: u16, 
}

#[derive(Debug, Serialize)]
enum UrbStatus {
    Success, 
    #[serde(with = "hex_serde")]
    Err(i32), 
}

fn parse_packet(packet: EnhancedPacketBlock) -> Packet {
    let urb_id = u64::from_le_bytes(packet.data[0..8].try_into().unwrap());
    
    let urb_type = match packet.data[8] {
        0x53 => UrbType::Submit,
        0x43 => UrbType::Complete,
        _    => UrbType::UrbError,
    };

    let transfer_type = match packet.data[9] {
        0x00 => TransferType::Isochronous, 
        0x01 => TransferType::Interrupt, 
        0x02 => TransferType::Control, 
        0x03 => TransferType::Bulk,
        _    => TransferType::ParsingError(packet.data[9]), 
    };

    let usb_endpoint = UsbEndpoint {
        raw: packet.data[10], 
        direction: if packet.data[10] & 0x80 != 0 {
            CommDirection::HostIn
        } else {
            CommDirection::HostOut
        }
    };

    let bus_number = u16::from_le_bytes(packet.data[12..14].try_into().unwrap());

    let setup_fragment = match packet.data[14] {
        0x00 => Some(SetupFragment {
            bmRequestType: packet.data[40], 
            bRequest: packet.data[41], 
            wValue: u16::from_le_bytes(packet.data[42..44].try_into().unwrap()),
            wIndex: u16::from_le_bytes(packet.data[44..46].try_into().unwrap()), 
            wLength: u16::from_le_bytes(packet.data[46..48].try_into().unwrap()), 
        }),
        _    => None,
    };
    let data_fragment = match packet.data[15] {
        0x00 => Some(parse_data_fragment(packet.clone())),
        _    => None,
    };

    let urb_status_value = i32::from_le_bytes(packet.data[28..32].try_into().unwrap()); 
    let urb_status = match urb_status_value {
        0 => UrbStatus::Success,
        e => UrbStatus::Err(e),
    };

    let transfer_flags = u32::from_le_bytes(packet.data[56..60].try_into().unwrap());
    
    let packet = Packet {
        id: urb_id, 
        typ: urb_type, 
        transfer_type,
        endpoint: usb_endpoint, 
        device_address: packet.data[11], 
        bus_number,
        setup_fragment, 
        data_fragment,
        urb_status, 
        transfer_flags,
    };

    println!("{:#04x?}", packet);
    packet
    
}

#[derive(Debug, Serialize)]
enum DataFragment {
    Other(String), 
    SbProtocol(InstructionFamily), 
}


#[derive(Debug, Serialize)]
enum InstructionFamily {
    ParsingError(String),
    Global(String),                   // 0x26 
    WriteSingleFeature(Feature),      // 0x12 (?) 
    Status(DeviceStatus),             // 0x11 (?) 
    MaybeDeviceAck(Feature),          // 0x02
    Unknown(String),
}

#[derive(Debug, Serialize)]
enum DeviceStatus {
    ParsingError(u8),
    Request(Feature),
    Response(Feature),
}

#[derive(Serialize)]
struct Feature {
    #[serde(serialize_with = "serialize_family_hex", skip_serializing_if = "Option::is_none")]
    family: Option<FeatureFamily>, 
    #[serde(serialize_with = "serialize_name_hex", skip_serializing_if = "Option::is_none")]
    name: Option<FeatureName>, 
    #[serde(serialize_with = "serialize_feature_value")]
    value: Option<f32>,
    raw_bytes: String,
}

impl fmt::Debug for Feature {
    fn fmt(&self, f: &mut fmt::Formatter<'_>) -> fmt::Result {
        let value_str = match self.value {
            Some(v) => format!("{:#010x} ({:.1})", v.to_bits(), v),
            None => "None".to_string(),
        };

        f.debug_struct("Feature")
            .field("family", &self.family)
            .field("name", &self.name)
            .field("value", &format_args!("{}", value_str))
            .finish()
    }
}

#[derive(Debug, Serialize)]
enum FeatureFamily {
    #[serde(with = "hex_serde")]
    ParsingError(u8),
    Global, 
    Dsp,
    MaybeDeviceAckWriteSingleFeature,
}

#[derive(Debug, Serialize)]
pub enum FeatureName {
    #[serde(with = "hex_serde")]
    ParsingError(u8),
    // SBX
    SurroundToggle,
    SurroundLevel,
    DialogPlusToggle,
    DialogPlusLevel,
    SmartVolumeToggle,
    SmartVolumeLevel,
    SmartVolumeMode,       // 0=Normal, 0x40000000=Night, 0x3f800000=Loud
    CrystalizerToggle,
    CrystalizerLevel,
    // EQ
    EqToggle,
    EqPreAmp,
    Eq31Hz,
    Eq62Hz,
    Eq125Hz,
    Eq250Hz,
    Eq500Hz,
    Eq1kHz,
    Eq2kHz,
    Eq4kHz,
    Eq8kHz,
    Eq16kHz,
    // Bass
    BassToggle,
    BassLevel,
}


fn parse_data_fragment(packet: EnhancedPacketBlock) -> DataFragment {
    if packet.data.len() <= 64 {
        return DataFragment::Other(String::from("No Payload"));
    }
    
    let data: &[u8] = &packet.data[64..];

    let preview_len = std::cmp::min(data.len(), 8);
    let mut df_start: String = data[..preview_len].iter()
    // let formatted = data[1..].iter()
        .map(|b| format!("{:#04x}", b))
        .collect::<Vec<_>>()
        .join(", ");
    df_start.push_str(", ...");
    
    if data[0] != 0x5a { return DataFragment::Other(df_start) };

    let formatted = data.iter()
    // let formatted = data[1..].iter()
        .map(|b| format!("{:#04x}", b))
        .collect::<Vec<_>>()
        .join(", ");

    let instruction_family: InstructionFamily = match data[1] {
        0x26 => InstructionFamily::Global(format!("TODO: Parse; Raw: {:?}", formatted)), 
        0x12 => InstructionFamily::WriteSingleFeature(parse_single_feature_write(data)),
        0x11 => InstructionFamily::Status(    
            match data[2] {
                0x03 => {
                    let feature = Feature {
                        family: get_feature_family(data[4]),
                        name: get_feature_name(data[5]),
                        value: None,
                        raw_bytes: formatted,
                    };
                    DeviceStatus::Request(feature)
                },
                0x08 => {
                    let feature = Feature {
                        family: get_feature_family(data[5]),
                        name: get_feature_name(data[6]),
                        value: Some(f32::from_le_bytes(data[7..11].try_into().unwrap())),
                        raw_bytes: formatted,
                    };
                    DeviceStatus::Response(feature)
                },
                _    => DeviceStatus::ParsingError(data[2]),
            }
        ),
        0x02 => InstructionFamily::MaybeDeviceAck(parse_device_ack_feature(data)),
        _    => InstructionFamily::ParsingError(formatted),
    };

    DataFragment::SbProtocol(instruction_family)
}

fn get_feature_name(feature_byte: u8) -> Option<FeatureName> {
    let feature_name = match feature_byte {
        0x00 => FeatureName::SurroundToggle,
        0x01 => FeatureName::SurroundLevel,
        0x02 => FeatureName::DialogPlusToggle,
        0x03 => FeatureName::DialogPlusLevel,
        0x04 => FeatureName::SmartVolumeToggle,
        0x05 => FeatureName::SmartVolumeLevel,
        0x06 => FeatureName::SmartVolumeMode,
        0x07 => FeatureName::CrystalizerToggle,
        0x08 => FeatureName::CrystalizerLevel,
        0x09 => FeatureName::EqToggle,
        0x0a => FeatureName::EqPreAmp,
        0x0b => FeatureName::Eq31Hz,
        0x0c => FeatureName::Eq62Hz,
        0x0d => FeatureName::Eq125Hz,
        0x0e => FeatureName::Eq250Hz,
        0x0f => FeatureName::Eq500Hz,
        0x10 => FeatureName::Eq1kHz,
        0x11 => FeatureName::Eq2kHz,
        0x12 => FeatureName::Eq4kHz,
        0x13 => FeatureName::Eq8kHz,
        0x14 => FeatureName::Eq16kHz,
        0x18 => FeatureName::BassToggle,
        0x19 => FeatureName::BassLevel,
        _    => FeatureName::ParsingError(feature_byte),
    };

    Some(feature_name)
}

fn get_feature_family(family_byte: u8) -> Option<FeatureFamily> {
    let feature_family = match family_byte {
        0x0a => FeatureFamily::MaybeDeviceAckWriteSingleFeature,
        0x96 => FeatureFamily::Dsp, 
        _    => FeatureFamily::ParsingError(family_byte),
    };

    Some(feature_family)
}

fn parse_device_ack_feature(data: &[u8]) -> Feature {
    let formatted = data.iter()
        .map(|b| format!("{:#04x}", b))
        .collect::<Vec<_>>()
        .join(", ");

    let family = get_feature_family(data[2]);

    Feature {
        family: family,
        name: None,
        value: None, 
        raw_bytes: formatted,
    }
}  

fn parse_single_feature_write(data: &[u8]) -> Feature {
    let formatted = data.iter()
        .map(|b| format!("{:#04x}", b))
        .collect::<Vec<_>>()
        .join(", ");

    let family = get_feature_family(data[4]);

    let name = get_feature_name(data[5]);

    let value: f32 = f32::from_le_bytes(data[6..10].try_into().unwrap());
    
    Feature {
        family: family, 
        name: name, 
        value: Some(value), 
        raw_bytes: formatted,
    }
}

In [ ]:
use std::path::PathBuf;

let pcap_files: Vec<PathBuf> = std::fs::read_dir(".")
    .expect("Failed to read directory")
    .filter_map(|e| e.ok())
    .map(|e| e.path())
    .filter(|p| p.extension().and_then(|e| e.to_str()) == Some("pcapng"))
    .collect();

for in_path in pcap_files {
    let out_path = in_path.with_extension("json");
    println!("Processing: {:?} -> {:?}", in_path, out_path);

    let in_file = File::open(&in_path).expect("File must exist");
    let mut reader = PcapNgReader::new(in_file).unwrap();

    let skip_non_sb: bool = true;
    let mut parsed_capture: Vec<Packet> = Vec::new();

    while let Some(block) = reader.next_block() {
        let block = block.unwrap();

        match block {
            Block::EnhancedPacket(packet) => {
                if skip_non_sb {
                    let Some(sbx_byte) = packet.data.get(64) else {
                        continue
                    };
                    if sbx_byte != &0x5a {
                        continue
                    }
                }

                println!("===== packet ===== {:?} =====", packet.timestamp);
                println!("Original Length : {:?}", packet.original_len);
                println!("Actual Length   : {:?}", packet.data.len());

                let packet = parse_packet(packet.clone());
                parsed_capture.push(packet);

                println!("===== endpkt ===== ------------------ =====");
            },
            _ => {},
        }
    }

    let file = std::fs::File::create(&out_path).expect("Failed to create file");
    serde_json::to_writer_pretty(file, &parsed_capture).expect("Failed to write JSON");
}